# Table of Contents

### Stage 1 - Prep the data
* [Step 1.) Import necessary libraries and adjust display settings](#Step1)
* [Step 2.) Create a primary source dataframe to work from](#Step2)
        - Load in raw data
        - Filter year range
        - Normalize team names (due to teams changing names and/or locations)
        - Factor ties into the win and loss column (ties equal 0.5 win and 0.5 loss)
        - Derive year over year win/loss change
* [Step 3.) Derive a secondary source dataframe to contain divisional win/loss information](#Step3)
        - Create a dictionary of NFL Division:Teams to help determine collective divisional win/loss
        - Populate secondary source dataframe by iteratively grabbing cumulative win/loss by year
        
### Stage 2 - The fun part. Answer trivia questions by analyzing primary or secondary dataframes

* [Question 1.) Which divisions have had the single most dominant regular seasons in the last 10 years? How about the worst?](#Question1)
* [Question 2.) For each of the last 10 years, which single division had the highest collective win %?](#Question2)
* [Question 3.) Which division has had the most sustained success the last 10 years? Defined as the highest average cumulative win %](#Question3)
* [Question 4.) Which NFL team has had the highest average win % over the last 10 years?](#Question4)
* [Question 5.) Which NFL team had the biggest year-to-year bounce back in the last 10 years? How about fall off?](#Question5)
* [Question 6.) Which of the last 10 years had the most parity across the league?](#Question6)

### Stage 3 - Analyze the team statistics to see which statistics best predict seasonal win/loss outcome
* [Part 1.) Create a table containing each statistic and it's R^2 value](#Part1)
* [Question 7.) Using point differential to predict success, which teams most overperformed and underperformed?](#Question7)
* [Part 2.) Create and identify optimal multiple regression models by iteratively evaluating thousands of models](#Part2)

### Stage 4 - Dynamically Export dataframes of interest created in Stages 1-3 to their own Excel worksheets within the source Excel Workbook
* [Part 1.) Create the appropriately named worksheets within the source Excel Workbook](#Stage4Step1)
* [Part 2.) Write the dataframes and their data to the newly created worksheets](#Stage4Step2)


# Stage 1 - Prep the Data

## Import necessary libraries and adjust display settings <a class="anchor" id="Step1"></a>

In [1]:
import os as os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import itertools
from openpyxl import load_workbook
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:.4f}'.format

## Load in raw data, filter year range, normalize team names, factor ties into the win and loss column, and derive year over year win/loss change

In [2]:
# Read in raw data 
nfl_dataframe = pd.read_excel('NFL Team Dataset - Copy.xlsx')

# Filter dataset to just look at the last 10 years (leaving 2013 in temporarily to calculate Y/O for 2014)
years_of_interest = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
nfl_dataframe = nfl_dataframe.loc[nfl_dataframe['year'].isin(years_of_interest)]

# Address the issue of teams changing name/location by normalizing the data to the latest team location/name
replacement_dict = {'San Diego Chargers':'Los Angeles Chargers','Washington Redskins':'Washington Commanders','St. Louis Rams':'Los Angeles Rams','Oakland Raiders':'Las Vegas Raiders','Washington Football Team':'Washington Commanders'}
nfl_dataframe.replace(replacement_dict,inplace=True)

if len(nfl_dataframe['team'].unique())!=32: 
    raise Exception('At least one alternate team name was found remaining')
    
# Replace NaN with 0 in 'ties' column
nfl_dataframe['ties'] = nfl_dataframe['ties'].replace(np.nan,0)

# Address ties by treating a tie as 0.5 to both win and loss column
nfl_dataframe['wins'] = nfl_dataframe['wins']+nfl_dataframe['ties']/2
nfl_dataframe['losses'] = nfl_dataframe['losses']+nfl_dataframe['ties']/2

# Create and populate column showing year over year win/loss change
nfl_dataframe['win change from prior year'] = ''

years_of_interest = [2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
list_of_nfl_teams = nfl_dataframe.team.unique()

for year in years_of_interest:
    
    for team in list_of_nfl_teams:
        
        year_before = year-1
        
        current_year_win_value = nfl_dataframe.loc[(nfl_dataframe['year']==year)&(nfl_dataframe['team']==team)]['wins'].reset_index(drop=True)[0]
        
        year_before_win_value = nfl_dataframe.loc[(nfl_dataframe['year']==year_before)&(nfl_dataframe['team']==team)]['wins'].reset_index(drop=True)[0]
        
        change_in_wins = current_year_win_value - year_before_win_value
        
        nfl_dataframe.loc[(nfl_dataframe['year']==year)&(nfl_dataframe['team']==team),['win change from prior year']] = change_in_wins
        
# Filter out 2013 data now that it's no longer needed
years_of_interest = [2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
nfl_dataframe = nfl_dataframe.loc[nfl_dataframe['year'].isin(years_of_interest)]

# Create clearer column titles using dictionary replace
column_name_dict = {'win change from prior year':'Win Change From Prior Year','year':'Year','team':'Team Name','wins':'Games Won','losses':'Games Lost','win_loss_perc':'Win/Loss Percentage','points':'Points For','points_opp':'Points Against','points_diff':'Point Differential','mov':'Average Margin of Victory','g':'Games Played','total_yards':'Offensive Yards Gained','plays_offense':'Offensive Plays Ran','yds_per_play_offense':'Yards Per Play Offense','turnovers':'Team Turnovers Lost','fumbles_lost':'Team Fumbles Lost','first_down':'First Downs Gained','pass_cmp':'Passes Completed','pass_att':'Pass Attempts','pass_yds':'Passing Yards','pass_td':'Passing Touchdowns','pass_int':'Interceptions Thrown','pass_net_yds_per_att':'Net Yards Gained Per Pass Attempt','pass_fd':'Passing First Downs Gained','rush_att':'Rushing Attempts','rush_yds':'Rushing Yards','rush_td':'Rushing TDs','rush_yds_per_att':'Rushing Yards Per Attempt','rush_fd':'Rushing First Downs','penalties':'Penalties Committed','penalties_yds':'Penalty Yards Committed','pen_fd':'First Downs by Penalty','score_pct':'Percentage of Drives Ending in Score','turnover_pct':'Percentage of Drives Ending in Turnover','exp_pts_tot':'Expected Points Contributed by Offense','ties':'Ties'}
nfl_dataframe.rename(columns=column_name_dict,inplace=True)

# Append dataframe to list of dataframe
list_of_dataframes = []
list_of_dataframes.append(nfl_dataframe)
list_of_dataframe_names = []
list_of_dataframe_names.append('NFL Data Refined')


nfl_dataframe.reset_index(drop=True).head(25)

,Year,Team Name,Games Won,Games Lost,Win/Loss Percentage,Points For,Points Against,Point Differential,Average Margin of Victory,Games Played,...,Rushing Yards Per Attempt,Rushing First Downs,Penalties Committed,Penalty Yards Committed,First Downs by Penalty,Percentage of Drives Ending in Score,Percentage of Drives Ending in Turnover,Expected Points Contributed by Offense,Ties,Win Change From Prior Year
0,2014,New England Patriots,12.0000,4.0000,0.7500,468,313,155,NaN,16,...,3.9000,96,120,1080,37,45.3000,7.2000,181.1200,0.0000,0.0000
1,2014,Buffalo Bills,9.0000,7.0000,0.5630,343,289,54,NaN,16,...,3.7000,65,124,1031,22,32.5000,11.2000,-43.3300,0.0000,3.0000
2,2014,Miami Dolphins,8.0000,8.0000,0.5000,388,373,15,NaN,16,...,4.7000,99,81,635,36,37.6000,10.5000,74.7800,0.0000,0.0000
3,2014,New York Jets,4.0000,12.0000,0.2500,283,401,-118,NaN,16,...,4.5000,112,109,932,23,31.7000,11.8000,-44.9200,0.0000,-4.0000
4,2014,Pittsburgh Steelers,11.0000,5.0000,0.6880,436,368,68,NaN,16,...,4.1000,98,103,834,38,41.4000,11.5000,156.5800,0.0000,3.0000
5,2014,Cincinnati Bengals,10.5000,5.5000,0.6560,365,344,21,NaN,16,...,4.4000,122,102,812,27,34.0000,13.1000,13.8400,1.0000,-0.5000
6,2014,Baltimore Ravens,10.0000,6.0000,0.6250,409,302,107,NaN,16,...,4.5000,113,111,870,38,39.6000,9.9000,135.2400,0.0000,2.0000
7,2014,Cleveland Browns,7.0000,9.0000,0.4380,299,337,-38,NaN,16,...,3.6000,88,116,932,27,28.0000,10.9000,-21.1500,0.0000,3.0000
8,2014,Indianapolis Colts,11.0000,5.0000,0.6880,458,369,89,NaN,16,...,3.9000,80,105,848,40,39.5000,14.6000,83.7300,0.0000,0.0000
9,2014,Houston Texans,9.0000,7.0000,0.5630,372,307,65,NaN,16,...,3.9000,108,94,834,34,33.0000,11.3000,13.0100,0.0000,7.0000


## Generate a dictionary containing (division, teams) as (key, value) pairs <a class="anchor" id="Step3"></a>

In [3]:
# Create a list of teams for each of the 8 divisions
afc_east_teams = ['New England Patriots','New York Jets','Miami Dolphins','Buffalo Bills']
afc_north_teams = ['Baltimore Ravens','Cleveland Browns','Pittsburgh Steelers','Cincinnati Bengals']
afc_west_teams = ['Kansas City Chiefs','Las Vegas Raiders','Denver Broncos','Los Angeles Chargers']
afc_south_teams = ['Houston Texans','Jacksonville Jaguars','Indianapolis Colts','Tennessee Titans']
nfc_east_teams = ['Dallas Cowboys','Philadelphia Eagles','New York Giants','Washington Commanders','Washington Redskins'] 
nfc_west_teams = ['San Francisco 49ers','Los Angeles Rams','St.Louis Rams','Seattle Seahawks','Arizona Cardinals']
nfc_north_teams = ['Detroit Lions','Green Bay Packers','Minnesota Vikings','Chicago Bears']
nfc_south_teams = ['Tampa Bay Buccaneers','New Orleans Saints','Atlanta Falcons','Carolina Panthers']

# Create a dictionary of division -> teams using the lists created above
nfl_division_dict = {'AFC East':afc_east_teams,'AFC North':afc_north_teams,'AFC West':afc_west_teams,'AFC South':afc_south_teams,'NFC East':nfc_east_teams, 'NFC North':nfc_north_teams,'NFC West':nfc_west_teams,'NFC South':nfc_south_teams}

# Due to the dictionary values being a list of lists, flatten dictionary values into one list
list_of_dict_values = []

for list_ in list(nfl_division_dict.values()):
    for value in list_:
        list_of_dict_values.append(value)
    
## Check and see if all team names in the dataframe map to an existing dictionary value
team_names_not_found = []
        
for team_name in nfl_dataframe['Team Name'].unique():
    
    if team_name not in list_of_dict_values:
        team_names_not_found.append(team_name)
        
if len(team_names_not_found)>0:
    raise Exception(f'{team_names_not_found} was/were not found in the (division,team names) dictionary')

## Iterate through the existing dataframe to generate a new dataframe that contains Divisional Win/Loss info over the last 10 years

In [4]:
# Create an empty dataframe to append data to
nfl_year_division_df = pd.DataFrame(columns=['Division','Year','Collective Wins','Collective Losses','Collective Win %'])

# Create a list of nfl divisions using dictionary keys
nfl_division_list = nfl_division_dict.keys()

# For each year in the 2010s, determine each conferences win % and append data to dataframe created above
for year in years_of_interest:
    
    # In the given year, determine each conferences win % and append data to dataframe created above
    
    for division in nfl_division_list:
        
        temporary_df = nfl_dataframe.loc[(nfl_dataframe['Year']==year) & (nfl_dataframe['Team Name'].isin(nfl_division_dict[division]))]
        
        temp_wins_sum_amount = temporary_df['Games Won'].sum()

        temp_losses_sum_amount = temporary_df['Games Lost'].sum() 

        temp_win_percent = (temp_wins_sum_amount)/(temp_wins_sum_amount+temp_losses_sum_amount)
        
        nfl_year_division_df.loc[len(nfl_year_division_df.index)]=division,year,temp_wins_sum_amount,temp_losses_sum_amount,temp_win_percent
        
# Append dataframe to list of dataframe
list_of_dataframes.append(nfl_year_division_df)
list_of_dataframe_names.append('Division Win % by Year')
        
nfl_year_division_df.reset_index(drop=True).head(20)        

,Division,Year,Collective Wins,Collective Losses,Collective Win %
0,AFC East,2014,33.0000,31.0000,0.5156
1,AFC North,2014,38.5000,25.5000,0.6016
2,AFC West,2014,33.0000,31.0000,0.5156
3,AFC South,2014,25.0000,39.0000,0.3906
4,NFC East,2014,32.0000,32.0000,0.5000
5,NFC North,2014,35.0000,29.0000,0.5469
6,NFC West,2014,37.0000,27.0000,0.5781
7,NFC South,2014,22.5000,41.5000,0.3516
8,AFC East,2015,36.0000,28.0000,0.5625
9,AFC North,2015,30.0000,34.0000,0.4688


# Stage 2 - Answer Fun Trivia Questions by Analyzing Data

## Question 1: 

### Which divisions have had the single most dominant regular seasons in the last 10 years? How about the worst? <a class="anchor" id="Question1"></a>

In [5]:
nfl_year_division_question_1_part1_df = nfl_year_division_df.sort_values('Collective Win %', ascending=False)

# Append dataframe to list of dataframe
list_of_dataframes.append(nfl_year_division_question_1_part1_df)
list_of_dataframe_names.append('Division Dominance Desc.')

nfl_year_division_question_1_part1_df

,Division,Year,Collective Wins,Collective Losses,Collective Win %
68,NFC East,2022,44.0000,24.0000,0.6471
73,AFC North,2023,43.0000,25.0000,0.6324
20,NFC East,2016,39.5000,24.5000,0.6172
1,AFC North,2014,38.5000,25.5000,0.6016
46,NFC West,2019,38.5000,25.5000,0.6016
49,AFC North,2020,38.5000,25.5000,0.6016
18,AFC West,2016,38.0000,26.0000,0.5938
62,NFC West,2021,40.0000,28.0000,0.5882
31,NFC South,2017,37.0000,27.0000,0.5781
6,NFC West,2014,37.0000,27.0000,0.5781


In [6]:
nfl_year_division_question_1_part2_df = nfl_year_division_df.sort_values('Collective Win %', ascending=True)

# Append dataframe to list of dataframe
list_of_dataframes.append(nfl_year_division_question_1_part2_df)
list_of_dataframe_names.append('Division Dominance Asc.')

nfl_year_division_question_1_part2_df

,Division,Year,Collective Wins,Collective Losses,Collective Win %
7,NFC South,2014,22.5000,41.5000,0.3516
67,AFC South,2022,24.0000,44.0000,0.3529
52,NFC East,2020,23.5000,40.5000,0.3672
44,NFC East,2019,24.0000,40.0000,0.3750
22,NFC West,2016,24.0000,40.0000,0.3750
3,AFC South,2014,25.0000,39.0000,0.3906
11,AFC South,2015,25.0000,39.0000,0.3906
79,NFC South,2023,27.0000,41.0000,0.3971
12,NFC East,2015,26.0000,38.0000,0.4062
59,AFC South,2021,28.0000,40.0000,0.4118


## Question 2: 

### For each of the last 10 years, which single division had the highest collective win %? <a class="anchor" id="Question2"></a>

In [7]:
# List of highest win % indices by year
list_of_indices = []

# Iterate through year and identify the index for the division with the highest win %. Append the index to a list
for year in years_of_interest:
    
    index = nfl_year_division_df.loc[nfl_year_division_df['Year']==year]['Collective Win %'].idxmax()
                                            
    list_of_indices.append(index)

# Look at highest win % indices by year
nfl_year_division_question_2_df = nfl_year_division_df.loc[list_of_indices].sort_values('Year',ascending=True)

# Append dataframe to list of dataframe
list_of_dataframes.append(nfl_year_division_question_2_df)
list_of_dataframe_names.append('Most Dominant Divisions')

nfl_year_division_question_2_df

,Division,Year,Collective Wins,Collective Losses,Collective Win %
1,AFC North,2014,38.5000,25.5000,0.6016
8,AFC East,2015,36.0000,28.0000,0.5625
20,NFC East,2016,39.5000,24.5000,0.6172
31,NFC South,2017,37.0000,27.0000,0.5781
35,AFC South,2018,35.0000,29.0000,0.5469
46,NFC West,2019,38.5000,25.5000,0.6016
49,AFC North,2020,38.5000,25.5000,0.6016
62,NFC West,2021,40.0000,28.0000,0.5882
68,NFC East,2022,44.0000,24.0000,0.6471
73,AFC North,2023,43.0000,25.0000,0.6324


## Question 3: 

### Which division has had the most sustained success the last 10 years? Defined as the highest average cumulative win % <a class="anchor" id="Question3"></a>

In [8]:
division_average_win_percent_df = pd.DataFrame(columns=['Division','Average Collective Win %'])

for division in nfl_division_list: 

    win_percent = nfl_year_division_df.loc[nfl_year_division_df['Division']==division]['Collective Win %'].sum()/10
    
    division_average_win_percent_df.loc[len(division_average_win_percent_df.index)]=division,win_percent
    
nfl_year_division_question_3_df = division_average_win_percent_df.sort_values('Average Collective Win %',ascending=False)

# Append dataframe to list of dataframe
list_of_dataframes.append(nfl_year_division_question_3_df)
list_of_dataframe_names.append('Division Sustained Success')

nfl_year_division_question_3_df

,Division,Average Collective Win %
1,AFC North,0.5245
6,NFC West,0.5223
2,AFC West,0.5200
0,AFC East,0.5163
5,NFC North,0.5097
4,NFC East,0.4853
7,NFC South,0.4831
3,AFC South,0.4390


## Question 4: 

### Which NFL team has had the highest average win % over the last 10 years? 
<a class="anchor" id="Question4"></a>

In [9]:
team_average_win_percent_df = pd.DataFrame(columns=['Team','Average Season Win %'])

list_of_nfl_teams = nfl_dataframe['Team Name'].unique()

for team in list_of_nfl_teams: 

    win_percent = nfl_dataframe.loc[nfl_dataframe['Team Name']==team]['Win/Loss Percentage'].sum()*100/10
    
    team_average_win_percent_df.loc[len(team_average_win_percent_df.index)]=team,win_percent
    
teamteam_average_win_percent_question_4_df = team_average_win_percent_df.sort_values('Average Season Win %',ascending=False)

# Append dataframe to list of dataframe
list_of_dataframes.append(teamteam_average_win_percent_question_4_df)
list_of_dataframe_names.append('Team Dominance')

teamteam_average_win_percent_question_4_df

,Team,Average Season Win %
13,Kansas City Chiefs,71.7800
0,New England Patriots,63.5800
4,Pittsburgh Steelers,63.3400
20,Green Bay Packers,62.3500
28,Seattle Seahawks,61.2700
6,Baltimore Ravens,60.1300
16,Dallas Cowboys,59.9400
1,Buffalo Bills,59.8400
25,New Orleans Saints,58.4800
17,Philadelphia Eagles,57.2100


## Question 5: 

### Which NFL team had the biggest year-to-year bounce back in the last 10 years? How about fall off? <a class="anchor" id="Question5"></a>

In [10]:
nfl_dataframe_subset = nfl_dataframe[['Year','Team Name','Games Won','Games Lost','Win Change From Prior Year']].reset_index(drop=True).copy()
nfl_dataframe_subset.rename(str.title,axis='columns',inplace=True)
nfl_dataframe_subset_question_5_df = nfl_dataframe_subset.sort_values('Win Change From Prior Year',ascending=False)

# Append dataframe to list of dataframe
list_of_dataframes.append(nfl_dataframe_subset_question_5_df)
list_of_dataframe_names.append('Bounce Back or Fall Off')

nfl_dataframe_subset_question_5_df

,Year,Team Name,Games Won,Games Lost,Win Change From Prior Year
80,2016,Dallas Cowboys,13.0000,3.0000,9.0000
188,2019,San Francisco 49ers,13.0000,3.0000,9.0000
56,2015,Carolina Panthers,15.0000,1.0000,7.5000
134,2018,Cleveland Browns,7.5000,8.5000,7.5000
136,2018,Houston Texans,11.0000,5.0000,7.0000
104,2017,Jacksonville Jaguars,10.0000,6.0000,7.0000
9,2014,Houston Texans,9.0000,7.0000,7.0000
124,2017,Los Angeles Rams,11.0000,5.0000,7.0000
148,2018,Chicago Bears,12.0000,4.0000,7.0000
296,2023,Houston Texans,10.0000,7.0000,6.5000


## Question 6: 

### Which of the last 10 years had the most parity across the league? <a class="anchor" id="Question6"></a>

In [11]:
overall_parity_df = pd.DataFrame(columns=['Year','Highest Win %','Average Win %','Lowest Win %','Standard Deviation','Relative Standard Deviation'])

list_of_parity_dataframes = []

for year in years_of_interest:
    
    df_name = str(f'year_{year}_parity_df')
    
    # Create subset_df
    subset_df = nfl_dataframe.loc[nfl_dataframe['Year']==year][['Year','Team Name','Games Won','Games Lost','Win/Loss Percentage']].sort_values('Win/Loss Percentage',ascending=False)
    
    globals()[df_name] = subset_df
    
    # Append dataframe to list_of_dataframes
    list_of_parity_dataframes.append(globals()[df_name])
    
    # Calculate standard dev 
    subset_df['Win/Loss Percentage'] = subset_df['Win/Loss Percentage']*100
    std = subset_df['Win/Loss Percentage'].std()

    # Set mean = 50
    mean = 50

    # Calculate relative standard deviation
    rsd = (std*100)/mean

    # Pickout value for highest and lowest win %
    highest_win_percent = subset_df['Win/Loss Percentage'].max()
    lowest_win_percent = subset_df['Win/Loss Percentage'].min()
    
    # Set values in place on overall parity table
    overall_parity_df.loc[len(overall_parity_df.index)] = year,highest_win_percent,mean,lowest_win_percent,std,rsd

overall_parity_df['Year'] = overall_parity_df['Year'].astype(int)
overall_parity_df_question_6 = overall_parity_df.sort_values('Relative Standard Deviation',ascending=True)

# Append dataframe to list of dataframe
list_of_dataframes.append(overall_parity_df_question_6)
list_of_dataframe_names.append('League Parity')

overall_parity_df_question_6

,Year,Highest Win %,Average Win %,Lowest Win %,Standard Deviation,Relative Standard Deviation
9,2023,76.5000,50.0000,11.8000,16.1606,32.3211
7,2021,76.5000,50.0000,17.6000,16.9602,33.9203
4,2018,81.3000,50.0000,18.8000,18.0660,36.1321
8,2022,82.4000,50.0000,17.6000,18.3933,36.7866
1,2015,93.8000,50.0000,18.8000,19.0479,38.0958
5,2019,87.5000,50.0000,12.5000,19.8107,39.6215
0,2014,75.0000,50.0000,12.5000,19.8430,39.6860
2,2016,87.5000,50.0000,6.3000,20.0136,40.0272
3,2017,81.3000,50.0000,0.0000,20.0257,40.0514
6,2020,87.5000,50.0000,6.3000,21.6401,43.2801


# Stage 3 - Analyze the different team statistics to determine which most influence seasonal win/loss outcome

## Analyzing the amount each statistic impacts win percentage using coefficient of determination (r^2)  <a class="anchor" id="Part1"></a>

In [12]:
r_squared_df = pd.DataFrame(columns=['Variable','R-squared Value'])

list_of_variables = nfl_dataframe.columns

list_of_vars_to_remove = ['Team Name','Average Margin of Victory','Win/Loss Percentage','Games Lost','Games Won','Ties','Games','Year']

list_of_variables = [x for x in list_of_variables if x not in list_of_vars_to_remove]

for variable in list_of_variables:
    
    # Initiate linear regression model
    model = LinearRegression()

    # Define predictor and response variables
    X = nfl_dataframe[variable]
    Y = nfl_dataframe['Win/Loss Percentage']
    
    X = np.array(X).reshape(-1,1)
    Y = np.array(Y).reshape(-1,1)

    # Fit regression model
    model.fit(X, Y)

    # Calculate R-squared of regression model
    r_squared = model.score(X, Y)
    
    # Write results to dataframe created above
    
    r_squared_df.loc[len(r_squared_df.index)]=variable,r_squared
    
r_squared_df.sort_values('R-squared Value',ascending=False,inplace=True)

# Append dataframe to list of dataframe
list_of_dataframes.append(nfl_dataframe_subset_question_5_df)
list_of_dataframe_names.append('R-Square Values by Variable')

r_squared_df

,Variable,R-squared Value
2,Point Differential,0.8114
0,Points For,0.5751
25,Percentage of Drives Ending in Score,0.4764
1,Points Against,0.4250
27,Expected Points Contributed by Offense,0.3746
15,Net Yards Gained Per Pass Attempt,0.3658
4,Offensive Yards Gained,0.3322
28,Win Change From Prior Year,0.2937
13,Passing Touchdowns,0.2909
9,First Downs Gained,0.2894


## Creating and identifying the best multiple regression model with number of explanatory variables of size N <a class="anchor" id="Part2"></a>

In [13]:
# Create general function that takes the following arguments: a list, max number of list items you wish to combine at once 
# and generates a list of all unique combinations of variables

def unique_list_gen(a_list,max_num_to_combine) -> list: 

    """
    
    Returns a list containing all unique combinations generated from individual values within a given list.

    This function piggybacks utilizes itertools.combinations to create a list of all combinations of a given list's items 
    with combinations of chosen size N or less. 

    Parameters
    ----------
    a_list : list
        The list of items that you wish to combine.
        
    max_num_to_combine : int
        The max number of list items you wish to combine together at once.

    Returns
    -------
    list
        A list containing all unique combinations of size N or smaller using items from provided list.
        
    """
    
    master_list_of_unique_item_combinations = []
    
    while max_num_to_combine > 0: 
    
        combinations = list(itertools.combinations(a_list, max_num_to_combine))

        master_list_of_unique_item_combinations = master_list_of_unique_item_combinations + combinations

        max_num_to_combine = max_num_to_combine - 1
        
    # Convert iterable into a list of lists
    master_list_of_unique_item_combinations = [list(x) for x in master_list_of_unique_item_combinations]
    
    return master_list_of_unique_item_combinations

In [14]:
r_squared_df2 = pd.DataFrame(columns=['Number of Variables','Variable(s)','R-squared Value'])

list_of_variables = nfl_dataframe.columns

list_of_vars_to_remove = ['Year','Team Name','Average Margin of Victory','Win/Loss Percentage','Games Won','Games Lost','Ties','Games','Win Change From Prior Year','Passes Completed','Penalty Yards Committed','Passing Attempts','Penalties Committed','First Downs by Penalty','Rushing Yards Per Attempt']

list_of_variables = [x for x in list_of_variables if x not in list_of_vars_to_remove]

list_of_combos = unique_list_gen(list_of_variables,3)

for variable in list_of_combos:
    
    # Grab length of variable
    variable_length = len(variable)
    
    # Initiate linear regression model
    model = LinearRegression()

    # Define predictor and response variables
    X = nfl_dataframe[variable]
    Y = nfl_dataframe['Win/Loss Percentage']
    
    X = np.array(X).reshape(-1,variable_length)
    Y = np.array(Y).reshape(-1,1)

    # Fit regression model
    model.fit(X, Y)

    # Calculate R-squared of regression model
    r_squared = model.score(X, Y)
    
    # Write results to dataframe created above
    r_squared_df2.loc[len(r_squared_df2.index)]=variable_length,variable,r_squared
    
r_squared_df2.sort_values('R-squared Value',ascending=False,inplace=True)

# Append dataframe to list of dataframe
list_of_dataframes.append(r_squared_df2)
list_of_dataframe_names.append('Multiple Regression')

r_squared_df2

,Number of Variables,Variable(s),R-squared Value
619,3,"[Point Differential, Rushing Yards, Percentage...",0.8206
626,3,"[Point Differential, Rushing First Downs, Perc...",0.8201
608,3,"[Point Differential, Passing First Downs Gaine...",0.8199
584,3,"[Point Differential, Passing Touchdowns, Perce...",0.8198
509,3,"[Point Differential, Yards Per Play Offense, P...",0.8196
...,...,...,...
2032,1,[Team Fumbles Lost],0.0765
2029,1,[Offensive Plays Ran],0.0571
1840,2,"[Games Played, Pass Attempts]",0.0013
2034,1,[Pass Attempts],0.0010


## Question 7:

## Which teams most underperformed and overperformed relative to point differential in the last 10 years? <a class="anchor" id="Question7"></a>

In [15]:
# Define predictor and response variables
X = nfl_dataframe['Point Differential']
Y = nfl_dataframe['Win/Loss Percentage']

# Reshape the data like x -> [[x]] and y -> [[y]] for model
X = np.array(X).reshape(-1,1)
Y = np.array(Y).reshape(-1,1)

# Define Model and create Linear Regression Line
point_diff_model = LinearRegression().fit(X, Y)

# Predict values using model
point_diff_model.predict([[-163]])

# Create new column and pass through values of point differential
nfl_dataframe['Predicted Win/Loss Percentage Based Off Point Differential'] = nfl_dataframe['Point Differential']

# Apply function to newly created column to convert point differential values into predicted win/loss percentage
def point_diff_to_win_perc(point_diff_Value):
    return point_diff_model.predict([[point_diff_Value]])[0][0]

nfl_dataframe['Predicted Win/Loss Percentage Based Off Point Differential'] = nfl_dataframe['Predicted Win/Loss Percentage Based Off Point Differential'].apply(point_diff_to_win_perc)

# Generate new column that is the difference of predicted and actual values
nfl_dataframe['Difference between actual and predicted values'] = nfl_dataframe['Win/Loss Percentage'] - nfl_dataframe['Predicted Win/Loss Percentage Based Off Point Differential']

In [16]:
nfl_dataframe_of_interest = nfl_dataframe[['Year','Team Name','Win/Loss Percentage','Predicted Win/Loss Percentage Based Off Point Differential','Difference between actual and predicted values','Point Differential']]

prediction_df_question_7_ascending = nfl_dataframe_of_interest.sort_values('Difference between actual and predicted values', ascending=True)

# Append dataframe to list of dataframe
list_of_dataframes.append(prediction_df_question_7_ascending)
list_of_dataframe_names.append('Underperforming Teams')

prediction_df_question_7_ascending

,Year,Team Name,Win/Loss Percentage,Predicted Win/Loss Percentage Based Off Point Differential,Difference between actual and predicted values,Point Differential
571,2020,Atlanta Falcons,0.2500,0.4690,-0.2190,-18
529,2019,Dallas Cowboys,0.5000,0.6968,-0.1968,113
455,2017,Cleveland Browns,0.0000,0.1942,-0.1942,-176
576,2021,Buffalo Bills,0.6470,0.8377,-0.1907,194
577,2021,New England Patriots,0.5880,0.7768,-0.1888,159
527,2019,Los Angeles Chargers,0.3130,0.4864,-0.1734,-8
427,2016,Jacksonville Jaguars,0.1880,0.3577,-0.1697,-82
431,2016,Los Angeles Chargers,0.3130,0.4777,-0.1647,-13
379,2014,Tampa Bay Buccaneers,0.1250,0.2690,-0.1440,-133
607,2021,Seattle Seahawks,0.4120,0.5507,-0.1387,29


In [17]:
prediction_df_question_7_descending = nfl_dataframe_of_interest.sort_values('Difference between actual and predicted values', ascending=False)

# Append dataframe to list of dataframe
list_of_dataframes.append(prediction_df_question_7_descending)
list_of_dataframe_names.append('Overperforming Teams')

prediction_df_question_7_descending

,Year,Team Name,Win/Loss Percentage,Predicted Win/Loss Percentage Based Off Point Differential,Difference between actual and predicted values,Point Differential
628,2022,Minnesota Vikings,0.7650,0.4951,0.2699,-3
550,2020,Cleveland Browns,0.6880,0.4811,0.2069,-11
532,2019,Green Bay Packers,0.8130,0.6098,0.2032,63
589,2021,Las Vegas Raiders,0.5880,0.3872,0.2008,-65
429,2016,Las Vegas Raiders,0.7500,0.5542,0.1958,31
556,2020,Kansas City Chiefs,0.8750,0.6933,0.1817,111
541,2019,Seattle Seahawks,0.6880,0.5125,0.1755,7
381,2014,Arizona Cardinals,0.6880,0.5194,0.1686,11
602,2021,Atlanta Falcons,0.4120,0.2464,0.1656,-146
449,2017,Buffalo Bills,0.5630,0.4011,0.1619,-57


In [18]:
_2023_nfl_dataframe_of_interest = nfl_dataframe[['Year','Team Name','Win/Loss Percentage','Predicted Win/Loss Percentage Based Off Point Differential','Difference between actual and predicted values','Point Differential']]

_2023_nfl_dataframe_of_interest = _2023_nfl_dataframe_of_interest.loc[_2023_nfl_dataframe_of_interest['Year']==2023]

_2023_prediction_df_question_7_descending = _2023_nfl_dataframe_of_interest.sort_values('Difference between actual and predicted values', ascending=False)

# Append dataframe to list of dataframe
list_of_dataframes.append(_2023_prediction_df_question_7_descending)
list_of_dataframe_names.append('2023 Overperforming Teams')

_2023_prediction_df_question_7_descending.head(10)

,Year,Team Name,Win/Loss Percentage,Predicted Win/Loss Percentage Based Off Point Differential,Difference between actual and predicted values,Point Differential
657,2023,Philadelphia Eagles,0.6470,0.5090,0.1380,5
646,2023,Pittsburgh Steelers,0.5880,0.4655,0.1225,-20
658,2023,New York Giants,0.3530,0.2551,0.0979,-141
670,2023,Seattle Seahawks,0.5290,0.4342,0.0948,-38
660,2023,Detroit Lions,0.7060,0.6151,0.0909,66
645,2023,Cleveland Browns,0.6470,0.5594,0.0876,34
654,2023,Denver Broncos,0.4710,0.4029,0.0681,-56
659,2023,Washington Commanders,0.2350,0.1716,0.0634,-189
642,2023,New York Jets,0.4120,0.3490,0.0630,-87
650,2023,Indianapolis Colts,0.5290,0.4672,0.0618,-19


In [19]:
_2023_prediction_df_question_7_ascending = _2023_prediction_df_question_7_descending.sort_values('Difference between actual and predicted values', ascending=True)

# Append dataframe to list of dataframe
list_of_dataframes.append(_2023_prediction_df_question_7_ascending)
list_of_dataframe_names.append('2023 Underperforming Teams')

_2023_prediction_df_question_7_ascending.head(10)

,Year,Team Name,Win/Loss Percentage,Predicted Win/Loss Percentage Based Off Point Differential,Difference between actual and predicted values,Point Differential
656,2023,Dallas Cowboys,0.7060,0.8377,-0.1317,194
668,2023,San Francisco 49ers,0.7060,0.8359,-0.1299,193
655,2023,Los Angeles Chargers,0.2940,0.4098,-0.1158,-52
665,2023,New Orleans Saints,0.5290,0.6307,-0.1017,75
640,2023,Buffalo Bills,0.6470,0.7438,-0.0968,140
644,2023,Baltimore Ravens,0.7650,0.8533,-0.0883,203
667,2023,Carolina Panthers,0.1180,0.1872,-0.0692,-180
662,2023,Minnesota Vikings,0.4120,0.4690,-0.0570,-18
663,2023,Chicago Bears,0.4120,0.4672,-0.0552,-19
671,2023,Arizona Cardinals,0.2350,0.2829,-0.0479,-125


# Stage 4 - Export dataframes to Excel

In [20]:
## Create a copy of Template Sheet and Rename 
wb = load_workbook('NFL Team Dataset - Copy.xlsx')

for dataframe_name in list_of_dataframe_names:
    
    ## Assign desired sheet name
    desired_sheet_name = dataframe_name

    ## Create a Copy of Template Sheet
    wb.copy_worksheet(wb['Template Sheet'])

    ## Rename template sheet to desired
    wb['Template Sheet Copy'].title = desired_sheet_name
    

## Save changes to file
wb.save('NFL Team Dataset - Copy.xlsx')

In [21]:
index_count = 0

for dataframe in list_of_dataframes:
    
    print(index_count)
    
    ## DEFINE DESIRED SHEET NAME
    desired_sheet_name = list_of_dataframe_names[index_count]

    ## Write to newly created sheet 
    with pd.ExcelWriter('NFL Team Dataset - Copy.xlsx', engine='openpyxl', mode='a',if_sheet_exists='overlay') as writer:

        ## Write the new DataFrame to a new sheet
        dataframe.to_excel(writer, sheet_name=desired_sheet_name, index=False)
        
    ## Change index counter
    index_count = index_count + 1
    
## Save changes to file
writer.close

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


<bound method ExcelWriter.close of <pandas.io.excel._openpyxl.OpenpyxlWriter object at 0x00000167A84FEF70>>